# AlexNet

In [ ]:
BATCH_SIZE = 128
LR = 0.01
NUM_EPOCHS = 10

In [ ]:
from torch import nn


net = nn.Sequential(
    # 这里使用一个 11*11 的更大窗口来捕捉对象。
    # 同时，步幅为 4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于 LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为 2 来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是 LeNet 中的好几倍。使用 dropout 层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用 Fashion-MNIST，所以用类别数为 10，而非论文中的 1000
    nn.Linear(4096, 10)
)

In [ ]:
import torch


X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

In [ ]:
from datasets import load_data_fashion_mnist


train_iter, test_iter = load_data_fashion_mnist(BATCH_SIZE, resize=224)

In [ ]:
from accelerate import try_gpu
from trainer import train_v2


train_v2(net, train_iter, test_iter, NUM_EPOCHS, LR, try_gpu())